In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

# folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'
folder = 'data/gum/자동'
# folder = 'data/yeongsan1/자동'

# ## 한강
# ## 자동
# file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
#               ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
#               ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
# #자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
file_names = [['현도_2016.xlsx','현도_2017.xlsx','현도_2018.xlsx', '현도_2019.xlsx'],
              ['대청호_2016.xlsx','대청호_2017.xlsx','대청호_2018.xlsx', '대청호_2019.xlsx'],
              ['옥천천_2016.xlsx','옥천천_2017.xlsx','옥천천_2018.xlsx', '옥천천_2019.xlsx'],
              ['장계_2016.xlsx','장계_2017.xlsx','장계_2018.xlsx', '장계_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['나주_2016.xlsx','나주_2017.xlsx','나주_2018.xlsx', '나주_2019.xlsx'],
#               ['지석천_2016.xlsx','지석천_2017.xlsx','지석천_2018.xlsx', '지석천_2019.xlsx'],
#               ['용봉_2016.xlsx','용봉_2017.xlsx','용봉_2018.xlsx', '용봉_2019.xlsx'],
#               ['서창교_2016.xlsx','서창교_2017.xlsx','서창교_2018.xlsx', '서창교_2019.xlsx'],
#               ['우치_2016.xlsx','우치_2017.xlsx','우치_2018.xlsx', '우치_2019.xlsx']]
              

In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:', ':,28:45', ':,26:29']
nakdong = [':,2:', ':,2:12']
gum = [':,2:'] #, ':,28:45', ':,26:29']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = gum[0]
iloc_val

':,2:'

In [8]:
def make_columns(df):
    column_list = ['측정날짜', '측정소명', '수온', '수소이온농도','전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a']
    list_df = pd.DataFrame(columns=column_list)
    list_df
    df = df.drop(columns=df.columns.difference(column_list))
    new_column = list_df.columns.difference(df.columns)
#     print(new_column)
    if not new_column.empty :
        print("Make_columns")
        for i in range(new_column.shape[0]):
            df[new_column[i]] = pd.Series()
#     print('columns')
#     print(df.columns)
    return df


In [9]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [10]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
#             df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
            df_tmp = make_columns(df_tmp)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [11]:
def normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    
    return df_all, train_mean, train_std

In [12]:
def min_max_normalize(df):
    # normalize data
    df_all = pd.concat(df)
    
    train_min = df_all.min()
    train_max = df_all.max()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_min)/(train_max - train_min)       
    
    
    return df_all, train_min, train_max

In [13]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)
# df_all, train_mean, train_std = min_max_normalize(df)

현도_2016.xlsx
(8784, 24)
(8784, 24)
현도_2017.xlsx
(8760, 24)
(8760, 24)
현도_2018.xlsx
(8756, 24)
(8760, 24)
현도_2019.xlsx
(8760, 24)
(8760, 24)
대청호_2016.xlsx
(8088, 19)
(8784, 19)
대청호_2017.xlsx
(8736, 19)
(8760, 19)
대청호_2018.xlsx
(8760, 19)
(8760, 19)
대청호_2019.xlsx
(8760, 19)
(8760, 19)
옥천천_2016.xlsx
(8783, 10)
(8784, 10)
Make_columns
옥천천_2017.xlsx


/home/cjinw/work/tf-2/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  # This is added back by InteractiveShellApp.init_path()


(8760, 10)
(8760, 10)
Make_columns
옥천천_2018.xlsx
(8754, 10)
(8760, 10)
Make_columns
옥천천_2019.xlsx
(8760, 10)
(8760, 10)
Make_columns
장계_2016.xlsx
(8764, 17)
(8784, 17)
장계_2017.xlsx
(8760, 17)
(8760, 17)
장계_2018.xlsx
(8759, 17)
(8760, 17)
장계_2019.xlsx
(8760, 17)
(8760, 17)


In [14]:
train_mean, train_std

(수온          1.616900e+01
 수소이온농도      7.806489e+00
 전기전도도       1.856745e+02
 용존산소        9.814162e+00
 총유기탄소       2.957587e+00
 총질소         1.771791e+00
 총인          1.902386e-02
 클로로필-a      9.978119e+00
 Day sin     6.037102e-15
 Day cos     2.747704e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           8.406999
 수소이온농도       0.768682
 전기전도도       57.720177
 용존산소         2.961568
 총유기탄소        1.035289
 총질소          0.712306
 총인           0.020042
 클로로필-a      11.020416
 Day sin      0.707109
 Day cos      0.707109
 Year sin     0.707102
 Year cos     0.707117
 dtype: float64)

## 

In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               out_features = None, out_num_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example2', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example3', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0     6     1     0]
 [    1    14     2     6]
 [    3    22     3    20]
 ...
 [ 6403    27     8 20235]
 [ 6411     7     7 20262]
 [ 6418    30    10 20269]]
miss_data file saved


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,7.9,6.9,177.0,5.1,1.6,NaN,NaN,3.6,-1.407890e-12,1.000000,-0.002666,0.999996
1,7.8,6.9,177.0,5.0,1.6,NaN,NaN,3.5,2.588190e-01,0.965926,-0.001950,0.999998
2,7.8,6.9,177.0,4.9,1.6,NaN,NaN,3.6,5.000000e-01,0.866025,-0.001233,0.999999
3,7.8,6.9,177.0,4.9,1.6,NaN,NaN,3.6,7.071068e-01,0.707107,-0.000516,1.000000
4,7.8,6.9,177.0,4.9,1.6,NaN,NaN,3.5,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
35059,8.0,7.2,160.0,10.4,2.9,NaN,NaN,8.5,-9.659258e-01,0.258819,-0.005734,0.999984
35060,8.0,7.2,160.0,10.4,2.9,NaN,NaN,8.5,-8.660254e-01,0.500000,-0.005017,0.999987
35061,7.9,7.2,160.0,10.4,2.9,NaN,NaN,8.4,-7.071068e-01,0.707107,-0.004301,0.999991
35062,7.9,7.2,161.0,10.4,2.9,NaN,NaN,8.4,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 12])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  12301


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 96.3359 - disc_loss: 0.7196 - rmse: 0.8824 - val_loss: 0.9001
Epoch 2/2000
1/1 [==============================] - 0s 105ms/step - gen_loss: 91.5002 - disc_loss: 0.5234 - rmse: 1.0082 - val_loss: 0.6494
Epoch 3/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 68.1581 - disc_loss: 0.4853 - rmse: 0.9137 - val_loss: 0.6601
Epoch 4/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 65.5276 - disc_loss: 0.3934 - rmse: 0.9108 - val_loss: 0.7221
Epoch 5/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 49.7331 - disc_loss: 0.3789 - rmse: 0.7225 - val_loss: 0.5721
Epoch 6/2000
1/1 [==============================] - 0s 100ms/step - gen_loss: 49.3880 - disc_loss: 0.3994 - rmse: 0.8151 - val_loss: 0.5240
Epoch 7/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 34.5362 - disc_loss: 0.3889 - rmse: 0.5111 - val_loss: 0.5940
Epo

1/1 [==============================] - 0s 131ms/step - gen_loss: 8.6602 - disc_loss: 0.2988 - rmse: 0.5401 - val_loss: 0.2341
Epoch 60/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 6.4346 - disc_loss: 0.2929 - rmse: 0.4645 - val_loss: 0.4201
Epoch 61/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 4.4233 - disc_loss: 0.2915 - rmse: 0.3464 - val_loss: 0.2854
Epoch 62/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 7.9730 - disc_loss: 0.2926 - rmse: 0.4733 - val_loss: 0.2526
Epoch 63/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 7.7428 - disc_loss: 0.2933 - rmse: 0.5214 - val_loss: 0.2128
Epoch 64/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 7.0136 - disc_loss: 0.2857 - rmse: 0.6469 - val_loss: 0.2788
Epoch 65/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 6.4646 - disc_loss: 0.2956 - rmse: 0.4378 - val_loss: 0.2223
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 103ms/step - gen_loss: 2.7085 - disc_loss: 0.2414 - rmse: 0.1883 - val_loss: 0.2732
Epoch 118/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 6.1343 - disc_loss: 0.2525 - rmse: 0.2960 - val_loss: 0.2754
Epoch 119/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 7.1771 - disc_loss: 0.2502 - rmse: 0.3067 - val_loss: 0.2199
Epoch 120/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 7.6597 - disc_loss: 0.2432 - rmse: 0.3686 - val_loss: 0.3112
Epoch 121/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 6.9006 - disc_loss: 0.2420 - rmse: 0.3663 - val_loss: 0.2925
Epoch 122/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 7.0697 - disc_loss: 0.2451 - rmse: 0.3166 - val_loss: 0.2107
Epoch 123/2000
1/1 [==============================] - 0s 147ms/step - gen_loss: 3.4128 - disc_loss: 0.2490 - rmse: 0.2412 - val_loss: 0.3182
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 130ms/step - gen_loss: 5.9779 - disc_loss: 0.2109 - rmse: 0.2915 - val_loss: 0.1757
Epoch 176/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 4.9278 - disc_loss: 0.2162 - rmse: 0.2566 - val_loss: 0.2237
Epoch 177/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 5.7655 - disc_loss: 0.2288 - rmse: 0.3042 - val_loss: 0.2686
Epoch 178/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 4.6499 - disc_loss: 0.2165 - rmse: 0.2671 - val_loss: 0.2814
Epoch 179/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 3.5043 - disc_loss: 0.2162 - rmse: 0.2290 - val_loss: 0.2894
Epoch 180/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 2.0036 - disc_loss: 0.2127 - rmse: 0.1700 - val_loss: 0.2260
Epoch 181/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 2.0328 - disc_loss: 0.2076 - rmse: 0.1740 - val_loss: 0.1860
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 129ms/step - gen_loss: 3.7762 - disc_loss: 0.2071 - rmse: 0.1844 - val_loss: 0.2064
Epoch 234/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 3.2403 - disc_loss: 0.2019 - rmse: 0.1763 - val_loss: 0.1900
Epoch 235/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 2.5753 - disc_loss: 0.2045 - rmse: 0.1689 - val_loss: 0.1621
Epoch 236/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 4.2816 - disc_loss: 0.1982 - rmse: 0.2029 - val_loss: 0.2180
Epoch 237/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 3.4494 - disc_loss: 0.2062 - rmse: 0.1896 - val_loss: 0.1802
Epoch 238/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 4.9420 - disc_loss: 0.2010 - rmse: 0.2119 - val_loss: 0.1901
Epoch 239/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 3.4531 - disc_loss: 0.1995 - rmse: 0.1867 - val_loss: 0.1704
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 147ms/step - gen_loss: 2.8035 - disc_loss: 0.1242 - rmse: 0.2203 - val_loss: 0.1796
Epoch 1389/2000
1/1 [==============================] - 0s 150ms/step - gen_loss: 1.5986 - disc_loss: 0.1203 - rmse: 0.1614 - val_loss: 0.1516
Epoch 1390/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 2.1391 - disc_loss: 0.1178 - rmse: 0.2129 - val_loss: 0.1564
Epoch 1391/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 2.5794 - disc_loss: 0.1205 - rmse: 0.2295 - val_loss: 0.1945
Epoch 1392/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 2.2983 - disc_loss: 0.1197 - rmse: 0.2131 - val_loss: 0.1431
Epoch 1393/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 2.6270 - disc_loss: 0.1220 - rmse: 0.2367 - val_loss: 0.1421
Epoch 1394/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 2.6826 - disc_loss: 0.1199 - rmse: 0.2301 - val_loss: 0.1371
Epoch 1395/2000
1/1 [=

1/1 [==============================] - 0s 123ms/step - gen_loss: 1.7401 - disc_loss: 0.1167 - rmse: 0.1689 - val_loss: 0.1351
Epoch 1446/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 1.7711 - disc_loss: 0.1273 - rmse: 0.1887 - val_loss: 0.1345
Epoch 1447/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 4.8324 - disc_loss: 0.1188 - rmse: 0.3954 - val_loss: 0.1988
Epoch 1448/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 2.2595 - disc_loss: 0.1214 - rmse: 0.2274 - val_loss: 0.1684
Epoch 1449/2000
1/1 [==============================] - 0s 110ms/step - gen_loss: 1.6063 - disc_loss: 0.1209 - rmse: 0.1540 - val_loss: 0.1747
Epoch 1450/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 2.5842 - disc_loss: 0.1225 - rmse: 0.2492 - val_loss: 0.1622
Epoch 1451/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 2.9086 - disc_loss: 0.1171 - rmse: 0.2443 - val_loss: 0.1857
Epoch 1452/2000
1/1 [=

1/1 [==============================] - 0s 106ms/step - gen_loss: 2.4915 - disc_loss: 0.1175 - rmse: 0.2409 - val_loss: 0.1261
Epoch 1503/2000
1/1 [==============================] - 0s 113ms/step - gen_loss: 2.4489 - disc_loss: 0.1187 - rmse: 0.2020 - val_loss: 0.1717
Epoch 1504/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 1.8356 - disc_loss: 0.1203 - rmse: 0.2014 - val_loss: 0.1854
Epoch 1505/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 1.4063 - disc_loss: 0.1177 - rmse: 0.1328 - val_loss: 0.1373
Epoch 1506/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 1.6929 - disc_loss: 0.1206 - rmse: 0.1709 - val_loss: 0.1824
Epoch 1507/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 2.5872 - disc_loss: 0.1247 - rmse: 0.2989 - val_loss: 0.1688
Epoch 1508/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 1.6926 - disc_loss: 0.1205 - rmse: 0.1769 - val_loss: 0.1772
Epoch 1509/2000
1/1 [=

1/1 [==============================] - 0s 144ms/step - gen_loss: 1.5887 - disc_loss: 0.1169 - rmse: 0.1373 - val_loss: 0.3110
Epoch 1560/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 1.9948 - disc_loss: 0.1183 - rmse: 0.1602 - val_loss: 0.1325
Epoch 1561/2000
1/1 [==============================] - 0s 107ms/step - gen_loss: 2.2189 - disc_loss: 0.1141 - rmse: 0.1746 - val_loss: 0.2726
Epoch 1562/2000
1/1 [==============================] - 0s 163ms/step - gen_loss: 1.9649 - disc_loss: 0.1175 - rmse: 0.1462 - val_loss: 0.2159
Epoch 1563/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 1.6169 - disc_loss: 0.1165 - rmse: 0.1568 - val_loss: 0.2329
Epoch 1564/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 1.8878 - disc_loss: 0.1203 - rmse: 0.1418 - val_loss: 0.2519
Epoch 1565/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 1.9485 - disc_loss: 0.1165 - rmse: 0.1519 - val_loss: 0.3120
Epoch 1566/2000
1/1 [=

1/1 [==============================] - 0s 140ms/step - gen_loss: 2.2394 - disc_loss: 0.1129 - rmse: 0.1411 - val_loss: 0.2484
Epoch 1617/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 2.6223 - disc_loss: 0.1153 - rmse: 0.1799 - val_loss: 0.2095
Epoch 1618/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 1.5572 - disc_loss: 0.1157 - rmse: 0.1666 - val_loss: 0.1318
Epoch 1619/2000
1/1 [==============================] - 0s 121ms/step - gen_loss: 1.6559 - disc_loss: 0.1145 - rmse: 0.1411 - val_loss: 0.3668
Epoch 1620/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 1.6633 - disc_loss: 0.1147 - rmse: 0.1368 - val_loss: 0.3282
Epoch 1621/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 2.7533 - disc_loss: 0.1154 - rmse: 0.1428 - val_loss: 0.2354
Epoch 1622/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 1.7730 - disc_loss: 0.1173 - rmse: 0.1343 - val_loss: 0.2138
Epoch 1623/2000
1/1 [=

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 32ms/step - loss: 0.2178


0.2177969515323639

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

140256
(120, 12)
120
x.shape = (140160, 12)
x_reshape.shape = (1168, 120, 12)
169322
y_true.shape= (140160, 12)
169322


(1, 120, 12)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

37/37 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(140160, 12) (120, 12)
(140256, 12)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

140256
unit_shape= (120, 12)
120
n= 140160
(35040, 8)
(35040, 8)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소     총유기탄소       총질소        총인  \
0     -0.983586 -1.179277 -0.150285 -1.591779 -1.311312       NaN       NaN   
1     -0.995480 -1.179277 -0.150285 -1.625545 -1.311312       NaN       NaN   
2     -0.995480 -1.179277 -0.150285 -1.659311 -1.311312       NaN       NaN   
3     -0.995480 -1.179277 -0.150285 -1.659311 -1.311312       NaN       NaN   
4     -0.995480 -1.179277 -0.150285 -1.659311 -1.311312       NaN       NaN   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -0.840847 -0.919091 -0.531435 -0.950227 -0.538581 -0.892581 -0.649845   
35060 -0.840847 -0.919091 -0.548760 -0.916461 -0.538581 -0.854676 -0.550052   
35061 -0.852742 -0.919091 -0.548760 -0.916461 -0.538581 -0.832214 -0.649845   
35062 -0.864637 -0.919091 -0.548760 -0.882696 -0.538581 -0.872927 -0.550052   
35063 -0.852742 -0.919091 -0.548760 -0.916461 -0.538581 -0.882754 -0.550052   

         클로로필-a      

In [52]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
# print(oris[0].shape, gans[0].shape)
# print(oris[1].shape, gans[1].shape)
# print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

In [53]:
train_df 

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-0.983586,-1.179277,-0.150285,-1.591779,-1.311312,-0.726141,-0.434933,-0.578755,-1.999588e-12,1.414209,...,-0.749638,-0.124796,-1.423695,-0.268628,0.109589,1.109493,-1.214721,1.223082,-0.379052,-0.714866
1,-0.995480,-1.179277,-0.150285,-1.625545,-1.311312,-0.680600,-0.632913,-0.587829,3.660241e-01,1.366021,...,-0.749638,-0.086519,-1.423695,-0.268628,0.109589,1.109493,-1.311312,1.095328,-0.491341,-0.705792
2,-0.995480,-1.179277,-0.150285,-1.659311,-1.311312,-0.708140,-0.520848,-0.578755,7.071043e-01,1.224741,...,-0.699741,0.342865,-1.423695,-0.138535,0.109589,1.143259,-1.214721,1.074270,-0.372306,-0.723940
3,-0.995480,-1.179277,-0.150285,-1.659311,-1.311312,-0.795992,-0.718841,-0.578755,9.999964e-01,0.999996,...,-0.699741,-0.268477,-1.411800,-0.138535,0.109589,1.109493,-1.311312,1.114983,-0.421699,-0.733014
4,-0.995480,-1.179277,-0.150285,-1.659311,-1.311312,-0.526344,-0.553456,-0.587829,1.224741e+00,0.707104,...,-0.749638,-0.214735,-1.423695,-0.268628,0.109589,1.109493,-1.311312,1.152888,-0.399929,-0.714866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24523,0.348638,-1.049184,-0.929909,-1.659311,-0.248807,-0.498088,0.647463,-0.306533,-1.366021e+00,0.366024,...,0.358482,0.683532,0.431902,-0.398720,-0.600734,-0.308675,0.267345,-0.363314,0.098602,1.154392
24524,0.336743,-1.049184,-0.929909,-1.625545,-0.248807,-0.589341,0.697360,-0.297459,-1.224741e+00,0.707104,...,0.160280,0.726260,0.431902,-0.398720,-0.600734,-0.376207,0.136874,-0.373141,0.098602,1.090874
24525,0.336743,-1.049184,-0.929909,-1.693077,-0.248807,-0.543012,0.697360,-0.288385,-9.999964e-01,0.999996,...,0.413447,0.652235,0.420007,-0.528813,-0.600734,-0.443739,0.070949,-0.340852,0.148499,1.000133
24526,0.324848,-1.049184,-0.929909,-1.726843,-0.248807,-0.551436,0.697360,-0.297459,-7.071043e-01,1.224741,...,0.403409,0.608460,0.420007,-0.658906,-0.600734,-0.511271,0.175479,-0.152731,0.098602,0.963837


In [54]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [211]:
MAX_EPOCHS = 150
target_col = '클로로필-a'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,4,5,6,7]
out_num_features = len(out_features)
out_num_features

1

In [212]:
out_features

[7]

In [213]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [214]:
OUT_STEPS = 24*5

In [215]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [216]:
# train_df

In [217]:
# it = iter(wdg)

In [218]:
# x,y = next(it)
# x.shape, y.shape

In [219]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [220]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [221]:
def hour_to_day_mean(array):
    
#     print(array)
    
    time = 24
    array = tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)
#     print(array)
    
    return array

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = (input_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = (label_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#     label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = (predict_temp * (train_std[plot_col] + 1e-6)) + train_mean[plot_col]
#       predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [222]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [223]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    o = 0
    o1 = 0
    p = 0
    nse_sum1 = 0 
    nse_sum2 = 0
    pbias_sum1 = 0
    pbias_sum2 = 0
    
    
    predictions = model(inputs)
    

    
    predictions = predictions * train_std[plot_col] + train_mean[plot_col]
    labels = labels * train_std[plot_col] + train_mean[plot_col]
    
    
#     print(predictions)
    
#     print(labels)
    o1 = np.mean(labels.numpy())
#     print('o1')
#     print(o1)
    
    if min_max_normailze :
#         predictions = predictions * train_std[plot_col] + train_mean[plot_col]
#         labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        o = label_temp[4].numpy()
        p = pred_temp[4].numpy()
        
#         print('pred_temp')
#         print(pred_temp[4])
#         print('+++++++++++++')
#         print(label_temp[4])
        
#         print('p')
#         print(p)
#         print('o')
#         print(o)
        
        temp_m = o-p
#         print('temp')
#         print(temp_m)
#         print(o)

#         print(o-o1)
#         print('22222')
        
        nse_sum1 = nse_sum1 + temp_m**2
        nse_sum2 = nse_sum2 + (o-o1)**2
        
    
            
        pbias_sum1 = pbias_sum1 + temp_m
        pbias_sum2 = pbias_sum2 + o
            
#         pred_arr.append(pred_temp[4])
#         label_arr.append(label_temp[4])
    
    
#     print(nse_sum1)
#     print('111111')
#     print(nse_sum2)

#     print('nse_sum1')
#     print(nse_sum1)
#     print('nse_sum2')
#     print(nse_sum2)
#     print('pbias_sum1')
#     print(pbias_sum1)
#     print('pbias_sum2')
#     print(pbias_sum2)
    
        
    nse = 1 - (nse_sum1 / nse_sum2)
    pbias = (pbias_sum1 / pbias_sum2) * 100
    
    
    
    
    print('NSE')
    print(nse)
    
    print('PBIAS')
    print(pbias)
    
        
#     mae = mean_absolute_error(label_arr, pred_arr)
#     mse = mean_squared_error(label_arr, pred_arr)
#     rmse = np.sqrt(mse)
#     mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
#     print("mae:")  
#     print(mae)
    
#     print("mse:")  
#     print(mse)
    
#     print("rmse")  
#     print(rmse)
    
#     print("mape")  
#     print(mape)
    
    

WindowGenerator.compa = compa

In [224]:
# np.sqrt(-0.28892756)

In [225]:
train_df.shape

(24528, 36)

In [226]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [227]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [228]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [229]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [230]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               out_features = out_features,
                               out_num_features = out_num_features,
                               )

In [231]:
multi_val_performance = {}
multi_performance = {}

**Elman_RNN**

In [232]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [233]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [234]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 901ms/step - loss: 0.2151 - mean_absolute_error: 0.4435 - val_loss: 0.3367 - val_mean_absolute_error: 0.5582
Epoch 2/150
1/1 [==============================] - 0s 291ms/step - loss: 0.1744 - mean_absolute_error: 0.3894 - val_loss: 0.2939 - val_mean_absolute_error: 0.5142
Epoch 3/150
1/1 [==============================] - 0s 283ms/step - loss: 0.1599 - mean_absolute_error: 0.3777 - val_loss: 0.2529 - val_mean_absolute_error: 0.4809
Epoch 4/150
1/1 [==============================] - 0s 295ms/step - loss: 0.1220 - mean_absolute_error: 0.3175 - val_loss: 0.1877 - val_mean_absolute_error: 0.3984
Epoch 5/150
1/1 [==============================] - 0s 275ms/step - loss: 0.0852 - mean_absolute_error: 0.2655 - val_loss: 0.1374 - val_mean_absolute_error: 0.3322
Epoch 6/150
1/1 [==============================] - 0s 286ms/step - loss: 0.0633 - mean_absolute_error: 0.2241 - val_loss: 0.0772 - val_mean_absolute_error: 0.2434
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 313ms/step - loss: 0.0211 - mean_absolute_error: 0.1094 - val_loss: 0.0786 - val_mean_absolute_error: 0.2400
Epoch 52/150
1/1 [==============================] - 0s 299ms/step - loss: 0.0194 - mean_absolute_error: 0.1017 - val_loss: 0.0717 - val_mean_absolute_error: 0.2236
Epoch 53/150
1/1 [==============================] - 0s 277ms/step - loss: 0.0216 - mean_absolute_error: 0.1083 - val_loss: 0.0641 - val_mean_absolute_error: 0.2211
Epoch 54/150
1/1 [==============================] - 0s 299ms/step - loss: 0.0194 - mean_absolute_error: 0.1016 - val_loss: 0.0718 - val_mean_absolute_error: 0.2301
Epoch 55/150
1/1 [==============================] - 0s 284ms/step - loss: 0.0195 - mean_absolute_error: 0.1017 - val_loss: 0.0680 - val_mean_absolute_error: 0.2217
Epoch 56/150
1/1 [==============================] - 0s 319ms/step - loss: 0.0178 - mean_absolute_error: 0.0977 - val_loss: 0.0762 - val_mean_absolute_error: 0.2387
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 322ms/step - loss: 0.0151 - mean_absolute_error: 0.0826 - val_loss: 0.0614 - val_mean_absolute_error: 0.2195
Epoch 102/150
1/1 [==============================] - 0s 354ms/step - loss: 0.0118 - mean_absolute_error: 0.0800 - val_loss: 0.0591 - val_mean_absolute_error: 0.2168
Epoch 103/150
1/1 [==============================] - 0s 308ms/step - loss: 0.0152 - mean_absolute_error: 0.0965 - val_loss: 0.0703 - val_mean_absolute_error: 0.2371
Epoch 104/150
1/1 [==============================] - 0s 330ms/step - loss: 0.0140 - mean_absolute_error: 0.0846 - val_loss: 0.0674 - val_mean_absolute_error: 0.2356
Epoch 105/150
1/1 [==============================] - 0s 301ms/step - loss: 0.0124 - mean_absolute_error: 0.0816 - val_loss: 0.0553 - val_mean_absolute_error: 0.2070
Epoch 106/150
1/1 [==============================] - 0s 322ms/step - loss: 0.0171 - mean_absolute_error: 0.0906 - val_loss: 0.0527 - val_mean_absolute_error: 0.2008
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 94ms/step - loss: 0.0292 - mean_absolute_error: 0.1277
val performance = [0.06951530277729034, 0.23784533143043518]
test performance =  [0.029247619211673737, 0.12768550217151642]


In [235]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [236]:
multi_window.plot2(elman_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [237]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [238]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 917ms/step - loss: 0.2091 - mean_absolute_error: 0.4322 - val_loss: 0.3393 - val_mean_absolute_error: 0.5519
Epoch 2/150
1/1 [==============================] - 0s 299ms/step - loss: 0.1881 - mean_absolute_error: 0.4120 - val_loss: 0.3298 - val_mean_absolute_error: 0.5555
Epoch 3/150
1/1 [==============================] - 0s 370ms/step - loss: 0.1765 - mean_absolute_error: 0.3917 - val_loss: 0.2861 - val_mean_absolute_error: 0.5051
Epoch 4/150
1/1 [==============================] - 0s 298ms/step - loss: 0.1725 - mean_absolute_error: 0.3912 - val_loss: 0.2658 - val_mean_absolute_error: 0.4884
Epoch 5/150
1/1 [==============================] - 0s 309ms/step - loss: 0.1363 - mean_absolute_error: 0.3419 - val_loss: 0.2283 - val_mean_absolute_error: 0.4519
Epoch 6/150
1/1 [==============================] - 0s 283ms/step - loss: 0.1082 - mean_absolute_error: 0.2965 - val_loss: 0.2090 - val_mean_absolute_error: 0.4338
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 301ms/step - loss: 0.0263 - mean_absolute_error: 0.1211 - val_loss: 0.0465 - val_mean_absolute_error: 0.1883
Epoch 52/150
1/1 [==============================] - 0s 359ms/step - loss: 0.0230 - mean_absolute_error: 0.1068 - val_loss: 0.0459 - val_mean_absolute_error: 0.1869
Epoch 53/150
1/1 [==============================] - 0s 297ms/step - loss: 0.0208 - mean_absolute_error: 0.1020 - val_loss: 0.0472 - val_mean_absolute_error: 0.1892
Epoch 54/150
1/1 [==============================] - 0s 323ms/step - loss: 0.0211 - mean_absolute_error: 0.1025 - val_loss: 0.0395 - val_mean_absolute_error: 0.1718
Epoch 55/150
1/1 [==============================] - 0s 384ms/step - loss: 0.0220 - mean_absolute_error: 0.1075 - val_loss: 0.0484 - val_mean_absolute_error: 0.1898
Epoch 56/150
1/1 [==============================] - 0s 312ms/step - loss: 0.0223 - mean_absolute_error: 0.1055 - val_loss: 0.0461 - val_mean_absolute_error: 0.1832
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 325ms/step - loss: 0.0190 - mean_absolute_error: 0.1040 - val_loss: 0.0531 - val_mean_absolute_error: 0.2016
Epoch 102/150
1/1 [==============================] - 0s 294ms/step - loss: 0.0157 - mean_absolute_error: 0.0864 - val_loss: 0.0533 - val_mean_absolute_error: 0.2036
Epoch 103/150
1/1 [==============================] - 0s 321ms/step - loss: 0.0151 - mean_absolute_error: 0.0833 - val_loss: 0.0496 - val_mean_absolute_error: 0.1937
Epoch 104/150
1/1 [==============================] - 0s 322ms/step - loss: 0.0219 - mean_absolute_error: 0.1045 - val_loss: 0.0558 - val_mean_absolute_error: 0.2075
Epoch 105/150
1/1 [==============================] - 0s 301ms/step - loss: 0.0186 - mean_absolute_error: 0.0985 - val_loss: 0.0471 - val_mean_absolute_error: 0.1865
Epoch 106/150
1/1 [==============================] - 0s 328ms/step - loss: 0.0133 - mean_absolute_error: 0.0841 - val_loss: 0.0466 - val_mean_absolute_error: 0.1855
Epoch 107/150
1/1 [=====

100/100 [==============================] - 10s 101ms/step - loss: 0.0230 - mean_absolute_error: 0.1106
val performance = [0.045613572001457214, 0.1862335205078125]
test performance =  [0.02301829867064953, 0.11063626408576965]


**LSTM**

In [239]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [240]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 0.2317 - mean_absolute_error: 0.4579 - val_loss: 0.3520 - val_mean_absolute_error: 0.5695
Epoch 2/150
1/1 [==============================] - 0s 234ms/step - loss: 0.1973 - mean_absolute_error: 0.4222 - val_loss: 0.3303 - val_mean_absolute_error: 0.5487
Epoch 3/150
1/1 [==============================] - 0s 250ms/step - loss: 0.1786 - mean_absolute_error: 0.3949 - val_loss: 0.2956 - val_mean_absolute_error: 0.5158
Epoch 4/150
1/1 [==============================] - 0s 245ms/step - loss: 0.1651 - mean_absolute_error: 0.3822 - val_loss: 0.2554 - val_mean_absolute_error: 0.4719
Epoch 5/150
1/1 [==============================] - 0s 241ms/step - loss: 0.1270 - mean_absolute_error: 0.3287 - val_loss: 0.2370 - val_mean_absolute_error: 0.4577
Epoch 6/150
1/1 [==============================] - 0s 247ms/step - loss: 0.1060 - mean_absolute_error: 0.2975 - val_loss: 0.1773 - val_mean_absolute_error: 0.3912
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 286ms/step - loss: 0.0239 - mean_absolute_error: 0.1124 - val_loss: 0.0544 - val_mean_absolute_error: 0.1984
Epoch 52/150
1/1 [==============================] - 0s 319ms/step - loss: 0.0210 - mean_absolute_error: 0.1071 - val_loss: 0.0550 - val_mean_absolute_error: 0.2014
Epoch 53/150
1/1 [==============================] - 0s 294ms/step - loss: 0.0228 - mean_absolute_error: 0.1077 - val_loss: 0.0594 - val_mean_absolute_error: 0.2092
Epoch 54/150
1/1 [==============================] - 0s 269ms/step - loss: 0.0223 - mean_absolute_error: 0.1110 - val_loss: 0.0623 - val_mean_absolute_error: 0.2182
Epoch 55/150
1/1 [==============================] - 0s 269ms/step - loss: 0.0286 - mean_absolute_error: 0.1225 - val_loss: 0.0592 - val_mean_absolute_error: 0.2091
Epoch 56/150
1/1 [==============================] - 0s 284ms/step - loss: 0.0217 - mean_absolute_error: 0.1078 - val_loss: 0.0550 - val_mean_absolute_error: 0.2001
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 270ms/step - loss: 0.0150 - mean_absolute_error: 0.0865 - val_loss: 0.0691 - val_mean_absolute_error: 0.2245
Epoch 102/150
1/1 [==============================] - 0s 258ms/step - loss: 0.0169 - mean_absolute_error: 0.0913 - val_loss: 0.0621 - val_mean_absolute_error: 0.2166
Epoch 103/150
1/1 [==============================] - 0s 262ms/step - loss: 0.0182 - mean_absolute_error: 0.0904 - val_loss: 0.0608 - val_mean_absolute_error: 0.2160
Epoch 104/150
1/1 [==============================] - 0s 249ms/step - loss: 0.0112 - mean_absolute_error: 0.0727 - val_loss: 0.0626 - val_mean_absolute_error: 0.2192
Epoch 105/150
1/1 [==============================] - 0s 292ms/step - loss: 0.0159 - mean_absolute_error: 0.0860 - val_loss: 0.0644 - val_mean_absolute_error: 0.2264
Epoch 106/150
1/1 [==============================] - 0s 244ms/step - loss: 0.0171 - mean_absolute_error: 0.0910 - val_loss: 0.0535 - val_mean_absolute_error: 0.2017
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 131ms/step - loss: 0.0730 - mean_absolute_error: 0.2366
val performance = [0.0730278342962265, 0.23661504685878754]
test performance =  [0.027896607294678688, 0.11895985901355743]


**CNN model**

In [241]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [242]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 0s 498ms/step - loss: 0.2041 - mean_absolute_error: 0.4282 - val_loss: 0.2466 - val_mean_absolute_error: 0.4656
Epoch 2/150
1/1 [==============================] - 0s 211ms/step - loss: 0.1278 - mean_absolute_error: 0.3293 - val_loss: 0.1070 - val_mean_absolute_error: 0.2963
Epoch 3/150
1/1 [==============================] - 0s 250ms/step - loss: 0.0466 - mean_absolute_error: 0.1861 - val_loss: 0.0341 - val_mean_absolute_error: 0.1520
Epoch 4/150
1/1 [==============================] - 0s 245ms/step - loss: 0.0682 - mean_absolute_error: 0.2019 - val_loss: 0.0286 - val_mean_absolute_error: 0.1377
Epoch 5/150
1/1 [==============================] - 0s 219ms/step - loss: 0.0623 - mean_absolute_error: 0.1992 - val_loss: 0.0376 - val_mean_absolute_error: 0.1633
Epoch 6/150
1/1 [==============================] - 0s 251ms/step - loss: 0.0319 - mean_absolute_error: 0.1379 - val_loss: 0.0736 - val_mean_absolute_error: 0.2239
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 248ms/step - loss: 0.0144 - mean_absolute_error: 0.0838 - val_loss: 0.0677 - val_mean_absolute_error: 0.2242
Epoch 52/150
1/1 [==============================] - 0s 267ms/step - loss: 0.0139 - mean_absolute_error: 0.0851 - val_loss: 0.0571 - val_mean_absolute_error: 0.2025
Epoch 53/150
1/1 [==============================] - 0s 254ms/step - loss: 0.0165 - mean_absolute_error: 0.0924 - val_loss: 0.0641 - val_mean_absolute_error: 0.2151
Epoch 54/150
1/1 [==============================] - 0s 237ms/step - loss: 0.0165 - mean_absolute_error: 0.0912 - val_loss: 0.0668 - val_mean_absolute_error: 0.2215
Epoch 55/150
1/1 [==============================] - 0s 235ms/step - loss: 0.0133 - mean_absolute_error: 0.0847 - val_loss: 0.0601 - val_mean_absolute_error: 0.2062
Epoch 56/150
1/1 [==============================] - 0s 251ms/step - loss: 0.0134 - mean_absolute_error: 0.0834 - val_loss: 0.0634 - val_mean_absolute_error: 0.2097
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 242ms/step - loss: 0.0118 - mean_absolute_error: 0.0816 - val_loss: 0.0645 - val_mean_absolute_error: 0.2152
Epoch 102/150
1/1 [==============================] - 0s 252ms/step - loss: 0.0134 - mean_absolute_error: 0.0800 - val_loss: 0.0608 - val_mean_absolute_error: 0.2062
Epoch 103/150
1/1 [==============================] - 0s 255ms/step - loss: 0.0136 - mean_absolute_error: 0.0842 - val_loss: 0.0581 - val_mean_absolute_error: 0.2053
Epoch 104/150
1/1 [==============================] - 0s 268ms/step - loss: 0.0092 - mean_absolute_error: 0.0674 - val_loss: 0.0502 - val_mean_absolute_error: 0.1868
Epoch 105/150
1/1 [==============================] - 0s 239ms/step - loss: 0.0144 - mean_absolute_error: 0.0810 - val_loss: 0.0533 - val_mean_absolute_error: 0.1970
Epoch 106/150
1/1 [==============================] - 0s 254ms/step - loss: 0.0130 - mean_absolute_error: 0.0731 - val_loss: 0.0448 - val_mean_absolute_error: 0.1807
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 157ms/step - loss: 0.0494 - mean_absolute_error: 0.1802
val performance = [0.049439866095781326, 0.18017463386058807]
test performance =  [0.024118518456816673, 0.10577302426099777]


In [243]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 138ms/step - loss: 0.0468 - mean_absolute_error: 0.1771
val performance = [0.04684244096279144, 0.17713025212287903]
test performance =  [0.01986679993569851, 0.09753597527742386]


**performance**

In [244]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [313]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.4093486894782882
PBIAS
-12.485640083306727


In [300]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.41190335856379157
PBIAS
-3.6112480457451914


In [324]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.31469463959912725
PBIAS
-8.245351975833344


In [329]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 클로로필-a
NSE
0.30937341895107295
PBIAS
-9.558317538107302


In [330]:
multi_window.plot24(gru_model, plot_col=out_features[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …